# 1. Linear Model 

## 1.1 OLS model

- The linear regresssion model : $Y = \beta_0 + \beta_1 X_1 + ... \beta_p X_p + \epsilon$
- OLS 
    - **Ordinary least squared (OLS)** is a type of linear least squares method for estimating the unkown parameters in a linear regression. 
    - All parameters of OLS model are unbiased estimators 
    - $E(\hat{\beta}^{OLS}) = \beta$
    - $Var(\hat{\beta}^{OLS}) ↓$
    - Problems in multiple linear regression 
        - OLS cannot be computed when n < p (high-dimensional data). 
        - OLS has relatively large variance. 

## 1.2 [Ex] Ordinary Least Squares(OLS) 

```R
set.seed(123)
n <- 100
# pp : pp means the number of samples 
pp <- c(10, 50, 80, 95, 97, 98, 99)
B <- matrix(0, 100, length(pp))


for (i in 1:100) {
  for (j in 1:length(pp)) {
    beta <- rep(0, pp[j])
    # beta1 == 1, beta0, beta2, ... betap -> 0 
    beta[1] <- 1
    x <- matrix(rnorm(n*pp[j]), n, pp[j])
    # x %*% beta is same as x[, 1]
    y <- x %*% beta + rnorm(n)
    g <- lm(y~x)
    B[i,j] <- g$coef[2]
  }
}
boxplot(B, col="orange", boxwex=0.6, ylab="Coefficient estimates",
        names=pp, xlab="The number of predictors", ylim=c(-5,5))
abline(h=1, col=2, lty=2, lwd=2)
apply(B, 2, mean)
apply(B, 2, var)
```

![](Img/OLS01.png)

- This means the mean of $\hat{\beta}^{OLS}$ is located 1 approximately. 
- If the number of variable is increasing(p is increasing), then the variance becomes higher. 

## 1.3 Three classes of solving problems 

- To solve the problem (variance become higher when the number of features is bigger), we need to make p lower than n. 
- **Subset Selection** : Identify a subset of the p predictors that we belive to be related to the response. 
- **Shrinkage** : Fit a model involving all p predictors, but the estimated coefficients are shrunken towards zero relative to the OLS estimates. 
- **Dimension Reduction** : Project the p predictors into a M-dimensional subspace. PCA is example. 

# 2. Subset Selection 

## 2.1 Best subsets regression

- The most direct approach is called **all subsets** or **best subsets regression**. 
- <font color="red">Fit for all possible subsets and then choose model based on some criterion that balances **training error** with **model size**.</font> 

Let's consider when we want to make best subset model using 10 features. First, we make null model $M_0$, which contains no predictors. Next, for k = 1, 2, ... p, repeat the steps following : 

- Fit all $(p, k)$ models that contain exactly k predictors. (This models must have the same counts of predictors) 
- Pick the best among these $(p, k)$ models, and call it $M_k$. Here best is defined as having the smallest RSS, or equivalently largest $R^2$. (We only use RSS or $R^2$ when we pick the best model among $(p, k)$ models) 
- Select a single best model from among $M_0, M_1, ..., M_p$ using CVE, $C_p$, AIC, BIC, or adjusted $R^2$. 

## 2.2 [Ex] Best subsets regression on Hitters 


```R
library(ISLR)
names(Hitters)
dim(Hitters)
sum(is.na(Hitters$Salary))
Hitters <- na.omit(Hitters)
dim(Hitters)
sum(is.na(Hitters))
library(leaps)
fit <- regsubsets(Salary ~ ., Hitters)
summary(fit)
sg <- summary(fit)
names(sg)
dim(sg$which)
sg$which
plot(fit)
plot(fit, scale="Cp")
```

![](Img/BSR01.png)

- regsubsets function calculate the best model among $(p, k)$ models. 
- The default values of parameter nbest and nvmax are 1 and 8 in each. 
- https://www.rdocumentation.org/packages/leaps/versions/3.1/topics/regsubsets

![](Img/BSR02.png)

- The number of parameters of best model is 8 while using metrics as $C_p$. 

# 3. Choosing the Optimal Model 

- Metrics $RSS$ and $R^2$ are only used in choosing best model among $(p, k)$ models. 
- To choose a model with low test error, we need to consider two ways. 
    - Indirectly estimate test error : $C_p$, AIC, BIC, and adjusted $R^2$
    - Directly estimate test error : CVE 
- Mallow's $C_p$ : $C_p = \frac{1}{n}(RSS + 2d\hat{\sigma}^2)$. Trade off between RSS and $2d\hat{\sigma}^2$. This metric has penalty on the number of parameters. 
- AIC : $AIC = -2log(L) + 2d$
- BIC : $BIC = \frac{1}{n}(RSS + log(n)d\hat{\sigma}^2)$. This metric has penalty on the number of sample size. (If n > 8, then BIC > $C_p$)
- Adjusted $R^2$ : scale RSS term by $\frac{RSS}{n-d-1}$ which d is the number of parameter and n is the number of sample size.

## 3.1 [Ex] Find the best model considering same model size (RSS, $R^2$)

```R
big <- regsubsets(Salary ~ ., data=Hitters, nvmax=19, nbest=10)
sg <- summary(big)
dim(sg$which)
sg.size <- as.numeric(rownames(sg$which))
table(sg.size)
sg.rss <- tapply(sg$rss, sg.size, min)
w1 <- which.min(sg.rss)
sg.rsq <- tapply(sg$rsq, sg.size, max)
w2 <- which.max(sg.rsq)
par(mfrow=c(1,2))
plot(1:19, sg.rss, type="b", xlab="Number of Predictors",
     ylab="Residual Sum of Squares", col=2, pch=19)
points(w1, sg.rss[w1], pch="x", col="blue", cex=2)
plot(1:19, sg.rsq, type="b", xlab="Number of Predictors",
     ylab=expression(R^2), col=2, pch=19)
points(w2, sg.rsq[w2], pch="x", col="blue", cex=2)
```

![](Img/metrics01.png)

## 3.2 [Ex] Find the best model considering different model size(AIC, BIC, Adjust $R^2$)

```R
sg.cp <- tapply(sg$cp, sg.size, min)
w3 <- which.min(sg.cp)
sg.bic <- tapply(sg$bic, sg.size, min)
w4 <- which.min(sg.bic)
sg.adjr2 <- tapply(sg$adjr2, sg.size, max)
w5 <- which.max(sg.adjr2)
par(mfrow=c(1,3))
plot(1:19, sg.cp, type="b", xlab ="Number of Predictors",
     ylab=expression(C[p]), col=2, pch=19)
points(w3, sg.cp[w3], pch="x", col="blue", cex=2)
plot(1:19, sg.bic, type="b", xlab ="Number of Predictors",
     ylab="Bayesian information criterion", col=2, pch=19)
points(w4, sg.bic[w4], pch="x", col="blue", cex=2)
plot(1:19, sg.adjr2, type="b", xlab ="Number of Predictors",
     ylab=expression(paste("Adjusted ", R^2)), col=2, pch=19)
points(w5, sg.adjr2[w5], pch="x", col="blue", cex=2)
```

![](Img/metrics02.png)

## 3.3 [Ex] Find the best model considering Validation set

- Consider $RSS$ and $R^2$ among models with same sample size : $M_k$
- Consider $C_p$, $AIC$, $BIC$, and $Adjust R^2$ among models with different sample size 
- To find best model among $M_k$ models, we divide sample size into train size and test size. (The test size shouldn't be trained to train model.) 

```R
library(ISLR)
library(leaps)
names(Hitters)
Hitters <- na.omit(Hitters)

# Train-test split : Bootstrap (66%, 33%) 
set.seed(1234)
train <- sample(c(TRUE, FALSE), nrow(Hitters), replace=TRUE) 
test <- (!train) 

# Training model 
# Consider RSS and R^2 among models with same sample size : find M_k 
g1 <- regsubsets(Salary ~ ., data=Hitters[train, ], nvmax=19) 
test.mat <- model.matrix(Salary~., data=Hitters[test, ]) 
val.errors <- rep(NA, 19)
# Calculating validation error 
for (i in 1:19) {
  coefi <- coef(g1, id=i) 
  pred <- test.mat[, names(coefi)] %*% coefi
  val.errors[i] <- mean((Hitters$Salary[test]-pred)^2) 
}
val.errors
w <- which.min(val.errors) 

plot(1:19, val.errors, type="l", col="red",
     xlab="Number of Predictors", ylab="Validation Set Error")
points(1:19, val.errors, pch=19, col="blue")
points(w, val.errors[w], pch="x", col="blue", cex=2)
```

![](Img/Subset_CV_set.png)

## 3.4 [Ex] Find the best model considering K-fold CV

- We can calculate K-fold CV by two ways. 
- Option 1 : Make error matrix of matrix(K, parameters) -> MAE -> apply(mat, 2, mean) 
- Option 2 : Make error matrix of matrix(n, parameters) -> E -> apply(mat, 2, mean) 

```R
library(ISLR)
names(Hitters)

## Define new "predict" function on regsubset
predict.regsubsets <- function(object, newdata, id, ...) {
  form <- as.formula(object$call[[2]])
  mat <- model.matrix(form, newdata)
  coefi <- coef(object, id=id)
  xvars <- names(coefi)
  mat[, xvars] %*% coefi
}

# KFold Train-test split : Bootstrap (66%, 33%) using 10 folds 
set.seed(1234)
K <- 10 
folds <- sample(rep(1:K, length=nrow(Hitters))) 

# Initialize error matrix of every fold 
cv.errors <- matrix(NA, K, 19, dimnames=list(NULL, paste(1:19))) 

# Repeat calculation in ever 8 folds
for (k in 1:K) { 
  train <- sample(c(TRUE, FALSE), nrow(Hitters), replace=TRUE) 
  test <- (!train) 
  
  # Training model 
  # Consider RSS and R^2 among models with same sample size : find M_k 
  fit <- regsubsets(Salary ~ ., data=Hitters[folds!=k, ], nvmax=19) 
  
  # Calculating validation error 
  for (i in 1:19) {
    pred <- predict(fit, Hitters[folds==k, ], id=i) 
    cv.errors[k, i] <- mean((Hitters$Salary[folds==k]-pred)^2) 
  }
} 
apply(cv.errors, 2, mean)
K.ERR <- apply(cv.errors, 2, mean)
ww <- which.min(K.ERR)

# Visualize the test error
plot(1:19, K.ERR, type="l", col="red",
     xlab="Number of Predictors", ylab="Cross-Validation Error")
points(1:19, K.ERR, pch=19, col="blue")
points(ww, K.ERR[ww], pch="x", col="blue", cex=2)
```

![](Img/Subset_Kfold.png)

## 3.5 [Ex] Find the best model considering One-Standard Error Rules 

- Considering CVE as $(\bar{X} - \frac{\hat{\sigma}}{\sqrt{n}}, \bar{X} + \frac{\hat{\sigma}}{\sqrt{n}})$
- Finding the best model roughly. 
- Refer the best model which mean of CVE is lower than the high upperbond of the optimized model. 

```R

# Train-test split 
set.seed(111)
n = nrow(Hitters)
folds <- sample(rep(1:K, length=n))
CVR.1se <- matrix(NA, n, 19)

# Train the model M_k on ith fold 
for (i in 1:K) {
  fit <- regsubsets(Salary~., Hitters[folds!=i, ], nvmax=19)
  # Calculate CVE of test sample 
  for (j in 1:19) {
    pred <- predict(fit, Hitters[folds==i, ], id=j)
    CVR.1se[folds==i, j] <- (Hitters$Salary[folds==i]-pred)^2
  }
}

# Calculate average based on One-Standard Error rule 
avg <- apply(CVR.1se, 2, mean)
se <- apply(CVR.1se, 2, sd)/sqrt(n)
PE <- cbind(avg - se, avg, avg + se)

# Visualize test error 
matplot(1:19, PE, type="b", col=c(1,2,1), lty=c(3,1,3), pch=20,
        xlab="Number of Predictors", ylab="Cross-Validation Error")
points(which.min(avg), PE[which.min(avg),2],
       pch="o",col="blue",cex=2)
up <- which(PE[,2] < PE[which.min(PE[,2]),3])
points(min(up), PE[min(up),2], pch="x", col="blue", cex=2) 
```

![](Img/Subset_One_Standard.png)

# 4. Variable Selection Methods 

- We cannot use subset selection model in $n << p$ (high demensional data). 
- So a shrinkage method is recommended.

## 4.1 Shrinkage

- Technique that constrains or regularizes the coeeficient estimates $\hat{\beta}$, that shrinks the $\hat{\beta}$ towards zero.
    - $E(\hat{\beta}^{sh}) \neq \beta$
- Shrinking the $\hat{\beta}$ can reduce variance 
    - $Var(\hat{\beta}^{OLS}) >> Var(\hat{\beta}^{sh})$
- Examples 
    - Ridge
    - Lasso 
    - Elastic Net : Ridge + Lasso

## 4.2 Ridge Regression

- $RSS + \lambda\sum_{j=1}^{p}\beta_j^2$
- where $\lambda >= 0$ is a tuning parameter.
- For a grid of $\lambda$ : $\lambda_{max} = \lambda_1 > ... > \lambda_m = \lambda_{min}$.
- The $l_2$-norm of $\hat{\beta}$ : $||\hat{\beta_{\lambda_1}}||^2 < ... < ||\hat{\beta_{\lambda_m}}||^2$

```R
library(glmnet)
# model.matrix convert original data into new matrix which form added with intercept.
x <- model.matrix(Salary~., Hitters)[, -1] 
y <- Hitters$Salary

# Grid Search for hyper parameter tuning lambda 
grid <- 10^seq(10, -2, length = 100) 
# Fit the ridge model : Default grid of lambda 
ridge.mod <- glmnet(x, y, alpha = 0, lambda=grid) 

# row for the number of parameters 
# col for the number of hyperparameter lambda 
dim(coef(ridge.mod)) 
ridge.mod$lambda 

# Calculate l_2 norm of each lambda 
# If lambda increase, l_2 norm decreaes 
# If lambda decreaes, l_2 norm increase 
ridge.mod$lambda[60]
coef(ridge.mod)[,60]
sqrt(sum(coef(ridge.mod)[-1, 60]^2))

# Calculate l2-norm based on lambda 
l2.norm <- apply(ridge.mod$beta, 2, function(t) sum(t^2))
x.axis <- cbind(log(ridge.mod$lambda), l2.norm)
colnames(x.axis) <- c("log.lambda", "L2.norm")
x.axis

# Visualize plot
par(mfrow=c(1,2))
plot(ridge.mod, "lambda", label=TRUE)
plot(ridge.mod, "norm", label=TRUE)
```

![](Img/ridge1.png)

## 4.3 Lasso Regression

- Ridge have disadvantages of including all p predictors in the final model. 
- What we want to do is variable selection. 
- Lasso shrinks $\hat{\beta}$ towards zero.
- $RSS + \lambda\sum_{j=1}^{p}|\beta_j|$
- The $l_1$-norm of $\hat{\beta}$ : $df(\hat{\beta}_{\lambda_1}) = 0 <  ... < df(\hat{\beta}_{\lambda_m}) = m$
    - $df$ is the number of variable which takes part in training model.

```R
# lasso when the value of argument alpha is 1. 
lasso.mod <- glmnet(x, y, alpha=1) 
# the number of default value of lambda is 100. 
# However, considering complexity of parameter, our model us 80 lambdas.   
dim(coef(lasso.mod)) 

# Find the degree of freedom matrix 
las <- cbind(lasso.mod$lambda, lasso.mod$df) 
colnames(las) <- c("lambda", "df") 
las

# Find the beta
# The sum of beta which is not zero is same with the value of df. 
dim(lasso.mod$beta)
apply(lasso.mod$beta, 2, function(t) sum(t!=0))
apply(lasso.mod$beta, 2, function(t) sum(abs(t)))

# Calculate l1-norm based on lambda 
l1.norm <- apply(lasso.mod$beta, 2, function(t) sum(abs(t)))
x.axis <- cbind(log(lasso.mod$lambda), l1.norm)
colnames(x.axis) <- c("log.lambda", "L1.norm")
x.axis

# Visualize plot
par(mfrow=c(1,2))
plot(lasso.mod, "lambda", label=TRUE)
plot(lasso.mod, "norm", label=TRUE)
```

![](Img/lasso1.png)

# 5. Selecting the Tuning Parameter

- Cross-validation provides a simple way to tackle this problem. 
- Choose a grid of $\lambda$ values, compute the CVE for each value of $\lambda$. 
- Can apply One-Standard rule to Lasso

## 5.1 [Ex] Validation Set

- alpha = 0 : ridge regression 
- alpha = 1 : lasso regression 

```R
# Make dataset 
library(glmnet)
library(ISLR) 
names(Hitters) 
Hitters <- na.omit(Hitters) 

set.seed(123)
x <- model.matrix(Salary~., Hitters)[, -1] 
y <- Hitters$Salary

# Train-Test Split
train <- sample(1:nrow(x), nrow(x)/3) 
test <- (-train) 
y.test <- y[test]

# Hyperparameter tuning 
grid <- 10^seq(10, -2, length=100) 
r1 <- glmnet(x[train, ], y[train], alpha=0, lambda=grid)
ss <- 0:(length(r1$lambda)-1) 
Err <- NULL

# Cross validation Error for test sample 
for (i in 1:length(r1$lambda)) { 
    r1.pred <- predict(r1, s=ss[i], newx=x[test, ])
    Err[i] <- mean((r1.pred - y.test)^2) 
} 
wh <- which.min(Err) 
lam.opt <- r1$lambda[wh] 

# Get full model with optimized hyperparmeter 
r.full <- glmnet(x, y, alpha=0, lambda=grid) 
r.full$beta[, wh] 
predict(r.full, type="coefficients", s=lam.opt) 
```

## 5.2 [Ex] K-fold Cross Validation

```R
set.seed(1234)
cv.r <- cv.glmnet(x, y, alpha=0, nfolds=10)
names(cv.r) 
# cvm : The mean value of cross validation -> CVE 
# cvsd : The standard deviation of cross validation -> One-standard error 
# cvup : The upperbound of CVE -> cvm + cvsd 
# cvlo : The lowerbound of CVE -> cvm - cvsd 
# lambda.min : The lambda which optimize input model 
# lambda.1se : The lambda which optimize imput model based on one-standard error 

cbind(cv.r$cvlo, cv.r$cvm, cv.r$cvup)
# Scatter plot based on One-Standard error 
# left vertix line : log(lambda.min) 
# right vertix line(more shrinked model) : log(lambda.1se) 
plot(cv.r) 

which(cv.r$lambda==cv.r$lambda.min)
which(cv.r$lambda==cv.r$lambda.1se)
# 100, 54 -> lambda.min < lambda.1se 

b.min <- predict(cv.r, type="coefficients", s=cv.r$lambda.min)
b.1se <- predict(cv.r, type="coefficients", s=cv.r$lambda.1se)

# calculate l1-norm
# calculate sum(b.min!=0) - 1 to get l2-norm 
cbind(b.min, b.1se)
c(sum(b.min[-1]^2), sum(b.1se[-1]^2))
# sum(b.min[-1]^2) > sum(b.1se[-1]^2) 
```

![](Img/ridge2.png)

# 6.1 Consider reality 

## 6.1 The Bias-Variance tradeoff

- If lambda axis increases to the right 
- Overfittng vs Underfitting 
- (Low bias + High variance) vs (High bias + Low variance)
- (l1-norm, l2-norm increase) vs (l1-norm, l2-norm decrease) 
- $\lambda$ decrease vs $\lambda$ increase

## 6.2 Comparison between Lasso and Ridge 

- If nonzero coefficient are large, ridge is better. 
- If nonzero coefficient are small, lasso is better. 
- In high-dimensional data where spares model is assummed, lasso perform better. 

# 7. Regularization Methods 

- Regularization methods are based on a penalized likelihood : 
- $Q_{\lambda}(\beta_0, \beta) = -l(\beta_0, \beta) + p_{\lambda}(\beta)$ 
- $(\hat{\beta_0}, \hat{\beta}) = arg min Q_{\lambda}(\beta_0, \beta)$ 
- Penalized likelihood for quantitive 
    - Linear regression model : $y_i = \beta_0 + x_i^T \beta + \epsilon_i$
    - l1-norm : $\lambda \sum(\hat{\beta}^2)$ 
    - l2-norm : $\lambda \sum|\hat{\beta}|$
    - $Q_{\lambda}(\beta_0, \beta) = -l(\beta_0, \beta) + p_{\lambda}(\beta) = \frac{1}{2}\sum_{i=1}^{n}(y_i - \beta_0 + x_i^T \beta)^2 +  p_{\lambda}(\beta)$
- Penalized likelihood for binary 
    - CVE based on deviance : $CVE = \frac{1}{n}\sum_{k=1}^K -2\sum_{i \in C_k}(y_i \log{\hat{p_i}^{[-k]} + (1 - y_i) \log{(1-\hat{p_i}^{[-k]})})} $ 
        - if type="response" : $p_i(\beta_0, \beta) = \frac{e^{\beta_0 + x_i^T \beta}}{1 + e^{\beta_0 + x_i^T \beta}}$
    - CVE based on classification error : $CVE = \frac{1}{n}\sum\sum I(y_i - \hat{y_i})^{[-k]}$ 

## 7.1 [Ex] Heart Data 

```R
# Importing Datasets 
library(glmnet)
url.ht <- "https://www.statlearning.com/s/Heart.csv"
Heart <- read.csv(url.ht, h=T)
summary(Heart)

# Preprocessing Data 
Heart <- Heart[, colnames(Heart)!="X"]
Heart[,"Sex"] <- factor(Heart[,"Sex"], 0:1, c("female", "male"))
Heart[,"Fbs"] <- factor(Heart[,"Fbs"], 0:1, c("false", "true"))
Heart[,"ExAng"] <- factor(Heart[,"ExAng"], 0:1, c("no", "yes"))
Heart[,"ChestPain"] <- as.factor(Heart[,"ChestPain"])
Heart[,"Thal"] <- as.factor(Heart[,"Thal"])
Heart[,"AHD"] <- as.factor(Heart[,"AHD"])
summary(Heart)
dim(Heart)
sum(is.na(Heart))
Heart <- na.omit(Heart)
dim(Heart)
summary(Heart)

# Train model 
## Logistic Regression
# Add dummy variable to original matrix x wihtout intercept 
x <- model.matrix(AHD ~., Heart)[,-1]
y <- Heart$AHD

# Train model 
g1 <- glmnet(x, y, family="binomial")

# Comparison lambda - coefficient, l1-norm - coefficient 
par(mfrow=c(1,2))
plot(g1, "lambda", label=TRUE)
plot(g1, "norm", label=TRUE)
df <- cbind(g1$lambda, g1$df)
colnames(df) <- c("lambda", "nonzero")
rownames(df) <- colnames(g1$beta)
df
```

![](Img/heart1.png)
- If lambda decreases, the number of active coefficients increase. 
- If l1-norm increase(lambda decrease), the number of active coefficeints increase.


```R
# Make evaluation : 10 folds 
set.seed(1234) 
K <- 10 
n <- length(y) 
fold <- sample(rep(1:K, length.out=n)) 
lam <- g1$lambda 
MSE <- matrix(0, n, length(lam)) 
for (i in 1:K) {
  test <- fold==i
  tran <- fold!=i 
  g <- glmnet(x[tran, ], y[tran], family="binomial", lambda=lam)
  prob <- predict(g, x[test, ], s=lam, type="response")
  yval <- y[test]=="Yes"
  MSE[test, ] <- -2*(yval*log(prob) + (1-yval)*log(1-prob))
}
CVE <- apply(MSE, 2, mean)

# Visualize CVE metrics 
par(mfrow=c(1,1))
plot(log(lam), CVE, type="b", xlab="log lambda", ylab="CV errors")
abline(v=log(lam)[which.min(CVE)], col="red", lty=2)

# Make final model : With full training set 
lam[which.min(CVE)]
coef(g, s=lam[which.min(CVE)])
g1 <- glmnet(x, y, family="binomial", lambda=lam[which.min(CVE)])
coef(g1)
```

![](Img/heart2.png)

- K-fold without using cv.glmnet. 
- MSE has been used for model evaluation metrics.
- model.matrix function convert original data into categorical encoded columns and intercepts. 
- To apply this metrx to glmnet, we need to input matrix without intercept.(beacuse inner argument of intercept is True).
- However, if we calculate prediction from the result of coef function, then we need to multiply with matrix with intercept. 

## 7.2 [Ex] Heart Data with automatic CV  

```R
set.seed(123)
par(mfrow=c(1,3))
# Error metric : Deviance 
# foldID shows the restricted fold -> don't need to use nfolds parameters
gcv1 <- cv.glmnet(x, y, family="binomial", lambda=lam,
nfolds=5, type.measure="deviance")
plot(gcv1)

# Error metric : Classification error 
gcv2 <- cv.glmnet(x, y, family="binomial", lambda=lam,
nfolds=5, type.measure="class")
plot(gcv2)

# Error metric : ROC-AUC score
gcv3 <- cv.glmnet(x, y, family="binomial", lambda=lam,
nfolds=5, type.measure="auc")
plot(gcv3)

c(gcv1$lambda.min, gcv1$lambda.1se)
c(gcv2$lambda.min, gcv2$lambda.1se)
c(gcv3$lambda.min, gcv3$lambda.1se)
g0 <- glmnet(x, y, family="binomial", lambda=lam)
coef(g0, s=c(gcv1$lambda.min, gcv2$lambda.min, gcv3$lambda.min))
coef(g0, s=c(gcv1$lambda.1se, gcv2$lambda.1se, gcv3$lambda.1se))
```

![](Img/heart3.png) 

- Three plots are written based on one-standard error. 
- The left vertical line is the value of gcvl\$lambda.min.
- The right vertical line is the value of gcvl\$labmda.1se.

# 8. Simulation Study : Prediction Performance 

- If $X$ has $n \times p (200 \times 2000)$ size, we need to find which variables are selected and which variable affects target most(coefficients). 
- So, we need to consider variable models predicting well. 
    - M1 : $\hat{\beta}^{lasso} + \lambda_{min}$ 
    - M2 : $\hat{\beta}^{lasso} + \lambda_{1se}$ 
    - M3 : $\hat{\beta}^{lasso} + \lambda_{min} + \hat{\beta}^{ols}$ 
    - M4 : $\hat{\beta}^{lasso} + \lambda_{1se} + \hat{\beta}^{ols}$ 
    - M5 : $\hat{\beta}^{ols}[1:20]$
- After training model and with their regression coefficients, choose the best model with the lowest $MSE_{test}$.

## 8.1 [Ex] Quantitative Outcome

**Generate $X$ and $Y$** 

```R
# install.packages('glmnet')
library(glmnet)

# Generate X matrix and Y vector
sim.fun <- function(n, p, beta, family=c("gaussian", "binomial")) {
  family <- match.arg(family)
  if (family=="gaussian") {
    x <- matrix(rnorm(n*p), n, p)
    y <- x %*% beta + rnorm(n)
  }
  else {
    x <- matrix(rnorm(n*p), n, p)
    xb <- x %*% beta
    z <- exp(xb) / (1+exp(xb))
    u <- runif(n)
    y <- rep(0, n)
    y[z > u] <- 1
  }
  list(x=x, y=y)
}

# Configure dimension of n, p and beta(모수)
set.seed(1234)
n <- 200
p <- 2000
beta <- rep(0, p)
beta[1:20] <- runif(20, -1, 1)

# Generate x and y 
sim <- sim.fun(n, p, beta)
x <- sim$x; y <- sim$y
```

**Regression Coefficients based on Model1 and Model2** 

```R
# Fit the lasso with two different lambda values
# b hat is inferred regression coefficients : M1, M2 
g <- cv.glmnet(x, y, alpha=1, nfolds = 10)
bhat1 <- coef(g, s="lambda.min")
bhat2 <- coef(g, s="lambda.1se")
# Check coefficients with value is not 0. 
wh1 <- which(as.matrix(bhat1)!=0)
w1 <- wh1[-1]-1
wh2 <- which(as.matrix(bhat2)!=0)
w2 <- wh2[-1]-1
```

**Regression Coefficients based on Model3, Model4, and Model5** 

```R
# Compute ordinary least square estimates (unbiased estimates)
bhat3 <- bhat4 <- bhat5 <- rep(0, p+1)
bhat3[wh1] <- lm(y ~ x[, w1])$coef
bhat4[wh2] <- lm(y ~ x[, w2])$coef
bhat5[1:21] <- lm(y ~ x[, 1:20])$coef
```

**Calculate Test Error based on MSE among Models** 

```R
set.seed(56789)
# Generate test sets
test <- sim.fun(n, p, beta)
xt <- cbind(1, test$x)
yt <- test$y

# Test set prediction errors of 6 coefficient estimates
mean((yt - xt %*% bhat1)^2) # lasso_lambda.min (M1)
mean((yt - xt %*% bhat2)^2) # lasso_lambda.1se (M2)
mean((yt - xt %*% bhat3)^2) # least square_lambda.min (M3)
mean((yt - xt %*% bhat4)^2) # least square_lambda.1se (M4)
mean((yt - xt %*% bhat5)^2) # least square_nonzero beta (M5)
mean((yt - xt %*% c(0, beta))^2) # true beta (M6)

# Calculate TE repeatedly
set.seed(1)
# Generate new test sets 100 times
K <- 100
pred <- matrix(NA, K, 6)
for (i in 1:K) {
    test <- sim.fun(n, p, beta)
    xt <- cbind(1, test$x)
    yt <- test$y

    pred[i, 1] <- mean((yt - xt %*% bhat1)^2)
    pred[i, 2] <- mean((yt - xt %*% bhat2)^2)
    pred[i, 3] <- mean((yt - xt %*% bhat3)^2)
    pred[i, 4] <- mean((yt - xt %*% bhat4)^2)
    pred[i, 5] <- mean((yt - xt %*% bhat5)^2)
    pred[i, 6] <- mean((yt - xt %*% c(0, beta))^2)
}

apply(pred, 2, mean)
boxplot(pred, col=c(2,2,4,4,3,"orange"), boxwex=0.6,
names=c("M1", "M2", "M3", "M4", "M5", "M6"),
ylab="Prediction Error")            
```

![](Img/Simulation1.png)
- We can see that lasso model didn't select accurate variables rather than ols model. 

## 8.2 [Ex] Binary Outcome 

**Generate $X$ and $Y$**

```R
set.seed(111)
n <- 200
p <- 2000
beta <- rep(0, p)
beta[1:20] <- runif(20, -1, 1)
sim <- sim.fun(n, p, beta, family="binomial")
x <- sim$x; y <- sim$y
```

**Classification Error Function** 

```R
## Classification Error
class.fun <- function(test.x, test.y, beta, k=0.5) {
    # xb : calculate x % coefficients
    xb <- test.x %*% beta
    # exb : inferred probability of xb 
    exb <- exp(xb) / (1 + exp(xb))
    y <- rep(0, length(test.y))
    y[as.logical(exb > k)] <- 1
    min(mean(test.y!=y), mean(test.y!=(1-y)))
}
```

**Training Models 1 ~ 5** 

```R
g <- cv.glmnet(x, y, alpha=1, nfolds = 10, family="binomial")
bhat1 <- coef(g, s="lambda.min")
bhat2 <- coef(g, s="lambda.1se")
wh1 <- which(as.matrix(bhat1)!=0)
wh2 <- which(as.matrix(bhat2)!=0)
bhat3 <- bhat4 <- bhat5 <- rep(0, p+1)
w1 <- wh1[-1]-1; w2 <- wh2[-1]-1
bhat3[wh1] <- glm(y ~ x[, w1], family="binomial")$coef
bhat4[wh2] <- glm(y ~ x[, w2], family="binomial")$coef
bhat5[1:21] <- glm(y ~ x[, 1:20], family="binomial")$coef
```

**Calculate Test Error based on Missclassification Rate** 

```R
# Generate test sets
set.seed(56789)
test <- sim.fun(n, p, beta, family="binomial")
xt <- cbind(1, test$x); yt <- test$y

# Prediction error comparison
class.fun(xt, yt, bhat1) # lasso_lambda.min (M1)
class.fun(xt, yt, bhat2) # lasso_lambda.1se (M2)
class.fun(xt, yt, bhat3) # least square_lambda.min (M3)
class.fun(xt, yt, bhat4) # least square_lambda.1se (M4)
class.fun(xt, yt, bhat5) # least square_nonzero beta (M5)
class.fun(xt, yt, c(0, beta)) # true beta (M6)

# Calculate TE repeatedly
set.seed(35791)

# Generate new test sets 100 times
K <- 100
pred <- matrix(NA, K, 6)
for (i in 1:K) {
    test <- sim.fun(n, p, beta, family="binomial")
    xt <- cbind(1, test$x)
    yt <- test$y

    pred[i, 1] <- class.fun(xt, yt, bhat1)
    pred[i, 2] <- class.fun(xt, yt, bhat2)
    pred[i, 3] <- class.fun(xt, yt, bhat3)
    pred[i, 4] <- class.fun(xt, yt, bhat4)
    pred[i, 5] <- class.fun(xt, yt, bhat5)
    pred[i, 6] <- class.fun(xt, yt, c(0, beta))
}

apply(pred, 2, mean)
boxplot(pred, col=c(2,2,4,4,3,"orange"), boxwex=0.6,
names=c("M1", "M2", "M3", "M4", "M5", "M6"),
ylab="Prediction Error")  
```

![](Img/Simulation2.png)

- With accurate variable selection(using ols model), we can get better test error. 

# 9. Regularization Methods 

- $Q_{\lambda}(\beta_0, \beta) = -l(\beta_0, \beta) + p_{\lambda}(\beta)$

## 9.1 The negative log-likelihood function

- Quantitative outcome: least square loss function
- Binary outcome: logistic likelihood
- Matched case-control outcome: conditional logistic likelihood
- Count outcome: Poisson likelihood
- Qualitative outcome: Multinomial likelihood
- Survival outcome: Cox partial likelihood

## 9.2 Type of Penalty Functions
- Convex penalty functions
    - Lasso (Tibshirani, JRSS, 1996)
    - Fused lasso (Tibshirani et al. JRSS, 2005)
    - Adaptive lasso (Zou, JASA, 2006)
    - Elastic-net (Zou and Hastie, JRSS, 2005)
- Non-convex penalty functions
    - lq-norm penalty with 0 < q < 1
    - Smoothly clipped absolute deviation (SCAD) (Fan and Li, JASA, 2005)
    - Minimax concave penalty (MCP) (Zhang, AOS, 2010)
- Group structure penalty functions
    - Group lasso (Yuan and Lin, JRSS, 2006)
    - Graph-constrained regularization (Li and Li, AOAS 2010)
    - Sparse group lasso (Simon et al. JGCS 2013)

# 10. Elastic-Net Regression 

- Lasso penalty function($l1$-norm) 
    - If $p > n$, lasso selects at most $n$ variables. 
    - Lasso is indifferent to highly correlated variables and tends to pick only one variable. 
- Ridge penalty function($l2$-norm) 
    - If cannot perform variable selection.
    - Shrinks correlated features to each other. 
- Elastic-net regularization 
    - Combine Lasso and Ridge 
    - $p_{\lambda}(\beta) = \lambda \alpha \sum_{j=1}^p |\beta_j| + \lambda(1 - \alpha)\sum_{j=1}^p \beta_j^2$ 
    - $\alpha$ : mixing proportion 
        - Lasso if $\alpha = 1$
        - Ridge if $\alpha = 0$
    - Minimize the penalized log-likelihood where tuning parameters $\alpha$ and $\lambda$ are fixed.
    
## 10.1 [Ex] Golub Data 

**Prepare Dataset golub** 

```R
# 1. importing package and dataset 

# Install bioconductor package ’golubEsets’
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("golubEsets")
library(golubEsets)
data(Golub_Merge)

# Golub data
?Golub_Merge
dim(Golub_Merge)
varLabels(Golub_Merge)
Golub_Merge$ALL.AML
table(Golub_Merge$ALL.AML)
Golub_Merge$Gender

# Expression data
dim(exprs(Golub_Merge))
head(exprs(Golub_Merge))
tail(exprs(Golub_Merge))

# Transpose the predictor x into n by p matrix
x <- t(as.matrix(exprs(Golub_Merge)))
y <- Golub_Merge$ALL.AML
dim(x)
sum(is.na(x))
```

**Training models : Ridge and Lasso** 

```R
# Model1 : Ridge regression 
g0 <- glmnet(x, y, alpha=0, family="binomial")
par(mfrow=c(1,2))
plot(g0, "lambda")
plot(g0, "norm")

# Model2 : Lasso regression 
g1 <- glmnet(x, y, alpha=1, family="binomial")
par(mfrow=c(1,2))
plot(g1, "lambda")
plot(g1, "norm")

# Cross Validation Lasso 
set.seed(12345)
gvc <- cv.glmnet(x, y, alpha=1, family="binomial", nfolds=5)
gvc$lambda.min
gvc$lambda.1se
plot(gvc)
fit1 <- coef(gvc, s="lambda.min")
fit2 <- coef(gvc, s="lambda.1se")
c(sum(as.matrix(fit1)!=0), sum(as.matrix(fit2)!=0))
w1 <- which(as.matrix(fit1)!=0)
w2 <- which(as.matrix(fit2)!=0)
data.frame(name=colnames(x)[w1], beta=fit1[w1])
data.frame(name=colnames(x)[w2], beta=fit1[w2])
```

![](Img/Elastic1.png)
![](Img/Elastic2.png)

- In Ridge regression, all variables are selected for training.
- In Lasso regression, only subset variables are selected for training.
- In Lasso with Cross Valdiation, we can select opmimized number of variables.

**Training model : Elastic-Net** 

```R
# Elastic-Net regression
ge <- glmnet(x, y, alpha=0.5, family="binomial")
ge$df
ge$lambda
plot(ge, "lambda")

# Elastic-Net regression with K-folds Cross Validation 
set.seed(111)
gecv <- cv.glmnet(x, y, alpha=0.5, family="binomial", nfolds=5)
plot(gecv)
fit3 <- coef(gecv, s="lambda.min")
fit4 <- coef(gecv, s="lambda.1se")
c(sum(as.matrix(fit3)!=0), sum(as.matrix(fit4)!=0))
w3 <- which(as.matrix(fit3)!=0)
w4 <- which(as.matrix(fit4)!=0)
data.frame(name=colnames(x)[w3], beta=fit3[w3])
data.frame(name=colnames(x)[w4], beta=fit4[w4])
```

![](Img/Elastic3.png)

- In Elastic-Net regression, the number of selected variables increases from 27 to 91. 

**Calculate Test Error among Models based on Classification Error** 

```R
# Seperate training sets and test sets
Golub_Merge$Samples
tran <- Golub_Merge$Samples < 39
test <- !tran
c(sum(tran), sum(test))

# Calculate Test error from Lasso  
set.seed(123)
g1 <- cv.glmnet(x, y, alpha=1, family="binomial", K=5,
subset=tran)
fit1 <- coef(g1, s="lambda.min")
fit2 <- coef(g1, s="lambda.1se")
c(sum(as.matrix(fit1)!=0), sum(as.matrix(fit2)!=0))
p1 <- predict(g1, x[test,], s="lambda.min", type="class")
p2 <- predict(g1, x[test,], s="lambda.1se", type="class")

# Confusion Matrix from 2 models 
table(p1, y[test])
table(p2, y[test])

# Calculate Test error from Elastic-Net
set.seed(1234)
g2 <- cv.glmnet(x, y, alpha=0.5, family="binomial", K=5,
subset=tran)
fit3 <- coef(g2, s="lambda.min")
fit4 <- coef(g2, s="lambda.1se")
c(sum(as.matrix(fit3)!=0), sum(as.matrix(fit4)!=0))
p3 <- predict(g2, x[test,], s="lambda.min", type="class")
p4 <- predict(g2, x[test,], s="lambda.1se", type="class")

# Confusion Matrix from 2 models 
table(p3, y[test])
table(p4, y[test])
```

- In model1(p1), there is no missclassified prediction.
- In model2(p2), there is only one missclassified prediction. 
- Using Elastic-Net regression, the number of selected variables are greater than using lasso. 

**50 Simulation replications** 

```R
set.seed(111)
K <- 50
ERR <- DF <- array(0, c(K, 4))
for (i in 1:K) {
    # Train-Test Split 
    tran <- as.logical(rep(0, 72))
    tran[sample(1:72, 38)] <- TRUE
    test <- !tran
    
    # Training model : lasso vs elastic-net 
    g1 <- cv.glmnet(x, y, alpha=1, family="binomial", K=5, subset=tran)
    g2 <- cv.glmnet(x, y, alpha=0.5, family="binomial", K=5, subset=tran)
    
    # Calculate probability of prediction 
    p1 <- predict(g1, x[test,], s="lambda.min", type="class")
    p2 <- predict(g1, x[test,], s="lambda.1se", type="class")
    p3 <- predict(g2, x[test,], s="lambda.min", type="class")
    p4 <- predict(g2, x[test,], s="lambda.1se", type="class")
    
    # Calculate Degree of freedom (The number of selected variables) 
    DF[i, 1] <- sum(coef(g1, s="lambda.min")!=0)
    DF[i, 2] <- sum(coef(g1, s="lambda.1se")!=0)
    DF[i, 3] <- sum(coef(g2, s="lambda.min")!=0)
    DF[i, 4] <- sum(coef(g2, s="lambda.1se")!=0)
    
    # Calculate Missclassification Error
    ERR[i, 1] <- sum(p1!=y[test])/sum(test)
    ERR[i, 2] <- sum(p2!=y[test])/sum(test)
    ERR[i, 3] <- sum(p3!=y[test])/sum(test)
    ERR[i, 4] <- sum(p4!=y[test])/sum(test)
}
apply(ERR, 2, mean)
DF
apply(DF, 2, var)
```

- The number of selected variables among models are in order Elastic-net + lambda.min, Elastic-net + lambda.1se, Lasso + lambda.min, Lasso + lambda.1se